In the previous tutorial, we introduced you to the basics of binary finite fields, but didn't really dive into the math or the implementation. In this tutorial, we're going to go deeper and actually walk through the mathematics of how binary fields actually work.

# What is “binary finite fields”?

Finite fields of order $2^m$ ($GF(2^m)$) are called binary fields or characteristic-two finite fields. They are of special interest because they are particularly efficient for implementation in hardware, or on a binary computer.

The elements of $GF(2^m)$ are binary polynomials, i.e. polynomials whose coefficients are either 0 or 1. There are $2^m$ such polynomials in the field and the degree of each polynomial is no more than $m-1$. Therefore, the elements can be represented as $m$-bit strings. Each bit in the bit string corresponding to the coefficient in the polynomial at the same position. For example, $GF(2^3)$ contains 8 element ${0,1,x,x+1,x^2,x^2+1,x^2+x,x^2+x+1}$. $x+1$ is actually $0x^2+1x+1$, so it can be represented as a bit string 011. Similarly, $x^2+x= 1x^2+1x+0$, so it can be represented as 110.

# How can we perform integer operations in $GF(2^m)$? Are they the same as integer operations in binary representation?

Integer operations in $GF(2^m)$ are a little different from regular operations. In the following we provide details of addition, subtraction, multiplication, and division operations for binary numbers in $GF(2^m)$.

## Addition/Subtraction:

In modulo 2 arithmetics, $1+1 \equiv 0~~mod~~2$, $1+0 \equiv 1~~mod~~2$, and $0+0 \equiv 0~~mod~~2$, which coincide with bit-XOR, i.e. $1 \oplus 1 = 0$, $1 \oplus 0 = 1$ and $0 \oplus 0 = 0$. Therefore, for binary polynomials, addition is simply bit-by-bit XOR. Also, in modulo 2 arithmetics, $-1 \equiv 1~~mod~~2$, so the result of subtraction of elements is the same as addition. This is the general form of addition and subtraction operations:

$A = a_{m-1} x^{m-1}+a_{m-2} x^{m-2}+\ldots+a_1 x^1+a_0$ where $a_i \in {0,1}$ for $i = 0, \ldots, m-1$

$B = b_{m-1} x^{m-1}+b_{m-2} x^{m-2}+\ldots+b_1 x^1+b_0$ where $b_i \in {0,1}$ for $i = 0, \ldots, m-1$

$A+B = A-B = (a_{m-1} \oplus b_{m-1})x^{m-1}+(a_{m-2} \oplus b_{m-2})x^{m-2}+\ldots+(a_{1} \oplus b_{1})x^{1}+(a_{0} \oplus b_{0})$

#### Example:

$(x^2+x+1)+(x^3+x^2+1)= x^3+2x^2+x+2= x^3+x$ since $2 \equiv 0~~mod~~2$. It can also be computed as $0111 \oplus 1101 = 1010$

$(x^2+x+1)-(x^3+x^2+1)= x^3+x$

In [0]:
import starks
from starks.modp import IntegersModP
from starks.polynomial import polynomials_over
from starks.finitefield import FiniteField

In [0]:
#GF(2^4)
p = 2
m = 4
Zp = IntegersModP(p)
polysOver = polynomials_over(Zp)

In [0]:
#reduction function p(x) = x^4+x+1
coefficients = [Zp(0)] * 5
coefficients[0] = Zp(1)
coefficients[1] = Zp(1)
coefficients[4] = Zp(1)
poly = polysOver(coefficients)
field = FiniteField(p, m, polynomialModulus=poly)

In [0]:
#A = x^2+x+1
A = field(polysOver([1,1,1]))

In [0]:
#B = x^3+x^2+1
B = field(polysOver([1,0,1,1]))

In [0]:
A+B

In [0]:
A-B

## Multiplication:

Multiplication of binary polynomials can be implemented as simple bit-shift and XOR, but is general form we can define multiplications as follows:

$A = a_{m-1} x^{m-1}+a_{m-2} x^{m-2}+\ldots+a_1 x^1+a_0$ where $a_i \in {0,1}$ for $i = 0, \ldots, m-1$

$B = b_{m-1} x^{m-1}+b_{m-2} x^{m-2}+\ldots+b_1 x^1+b_0$ where $b_i \in {0,1}$ for $i = 0, \ldots, m-1$

$A \times B = (a_{m-1} \cdot b_{m-1})x^{2m-2}+(a_{m-1} \cdot b_{m-2}+a_{m-2} \cdot b_{m-1})x^{2m-1}+\ldots+(a_0 \cdot b_1+a_1 \cdot b_0 ) x^1+(a_0 \cdot b_0)$


#### Example:
$(x^2+x+1) \times (x^3+x^2+1) = x^5+2x^4+2x^3+2x^2+x+1= x^5+x+1$ after reduction modulo 2. It can also be computed as $0111 \times 1101 = 110100 \oplus 11010 \oplus 1101 = 100011$


#### Note:

In $GF(2^m)$, when the degree of the result is more than $m-1$, it needs to be reduced modulo an irreducible/reduction polynomial $p(x)$ in degree $m$. This can be implemented as bit-shift and XOR. The general form of multiplication operation is update as follows:

$A \times B = (a_{m-1} \cdot b_{m-1})x^{2m-2}+(a_{m-1} \cdot b_{m-2}+a_{m-2} \cdot b_{m-1})x^{2m-1}+\ldots+(a_0 \cdot b_1+a_1 \cdot b_0 ) x^1+(a_0 \cdot b_0)~~mod~~p(x)$

Since the details of reduction procedure is a little complex, and there exists some libraries to perform this computation, we do not focus on details of its computation here. If you are interested to learn the details, we encourage you to study section 2.3.5 of \cite{link4}. We provide an example in the following to show how reduction decreases size of multiplication.

#### Example:
Reduction polynomial: $p(x)= x^4+x+1$

$(x^2+x+1) \times (x^3+x^2+1) = x^5+x+1 \equiv x^2+1~~mod~~p(x)$

In [0]:
A*B

## Division:

The division operation is implemented by multiply and modulo inverse operation. In means that we can implement division operation as follows:

$A = a_{m-1} x^{m-1}+a_{m-2} x^{m-2}+\ldots+a_1 x^1+a_0$ where $a_i \in {0,1}$ for $i = 0, \ldots, m-1$

$B = b_{m-1} x^{m-1}+b_{m-2} x^{m-2}+\ldots+b_1 x^1+b_0$ where $b_i \in {0,1}$ for $i = 0, \ldots, m-1$

$\frac{A}{B} = A \times B^{-1}~~mod~~p(x)$ where $p(x)$ is reduction polynomial

#### How can we perform modulo inverse operation?

There is a well-known algorithm for computing modulo inverse which is Extended Euclidean GCD algorithm. Since, the implementation of Extended Euclidean GCD algorithm is already available, it the following, we provide the high-level idea of how Extended Euclidean GCD can help us to find modulo inverse. If you are interested to learn more, we encourage you to study following link:

https://engineering.purdue.edu/kak/compsec/NewLectures/Lecture5.pdf

Given an $a$ that is relatively prime to $n$, we must obviously have $gcd(a, n) = 1$. Such $a$ and $n$ must satisfy the following constraint for some $x$ and $y$:

$xa+yn = 1$

Let’s now consider this equation modulo $n$. Since $y$ is an integer $yn~~mod~~n$ equals 0. Thus, it must be the case that, considered modulo $n$, $x$ equals $a^{-1}$, the multiplicative inverse of a modulo $n$. We extend this solution to work in $GF(2^m)$.

#### Example:

Reduction polynomial: $p(x) = x^4+x+1$

$\frac{x^2+x+1}{x^3+x^2+1} = (x^2+x+1) \times (x^3+x^2+1)^{-1} = (x^2+x+1) \times (x^2) \equiv (x^3+x^2+x+1) ~~mod~~p(x)$


In [ ]:
A/B

## Exercises

1. Define new polynomials `C` and `D`. Work out by hand what `C + D` would be. Verify your answer using code.
2. Work out by hand what `C * D` would be. Verify your answer using code.
3. (Extra Credit) Work out what `C / D` would be.